In [ ]:
!pip install tez
!pip install efficientnet-pytorch
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 13.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=721c9f1ebf3a4af3c5ab1f265c8c7d20ea67f78c6b54317d809fbbeba4df8234
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet-pytorch
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import torch
import tez
import torch.nn as nn
import transformers
from transformers import AdamW,get_linear_schedule_with_warmup
from sklearn import metrics

In [ ]:
from torch.optim.adam import Adam
class BERTDataset:
    def __init__(self,texts,targets,max_len=64) :
        self.texts = texts
        self.targets = targets
        self.tokenizer =  transformers.BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case =False)
        self.max_length = max_len

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = str(self.texts  [idx])
        inputs = self.tokenizer.encode_plus(text,
                                            None,
                                            add_special_tokens=True,
                                            max_length=self.max_length,padding="max_length",
                                            truncation=True)
        return {"ids":torch.tensor(inputs["input_ids"],dtype = torch.long),
                "mask":torch.tensor(inputs["attention_mask"],dtype = torch.long),
                "token_type_ids":torch.tensor(inputs["token_type_ids"],dtype = torch.long),
                "targets" : torch.tensor(self.targets[idx],dtype = torch.float)
        }
class TextModel(tez.Model):
  def __init__(self,num_clasees,num_train_steps):
    super().__init__()
    self.bert = transformers.BertModel.from_pretrained('bert-base-uncased',return_dict=False)
    self.bert_drop = nn.Dropout(0.3)
    self.out = nn.Linear(768,num_clasees)
    self.num_train_steps = num_train_steps
  
  def fetch_optimizer(self):
    opt = AdamW(self.parameters(),lr=1e-4)
    return opt

  def fetch_scheduler(self):
    sch = get_linear_schedule_with_warmup(self.optimizer,
                                          num_warmup_steps=0,
                                          num_training_steps=self.num_train_steps)
    return sch

  def loss(self,outputs,targets):
    return nn.BCEWithLogitsLoss()(outputs,targets.view(-1,1))

  def monitor_metrics(self,outputs,targets):
    outputs = torch.sigmoid(outputs).cpu().detach().numpy() >=0.5
    targets = targets.cpu().detach().numpy()
    return {
        "accuracy":metrics.accuracy_score(targets,outputs)
    }

  def forward(self,ids,mask,token_type_ids,targets=None):
    _,x = self.bert(ids,attention_mask = mask,token_type_ids=token_type_ids)
    x = self.bert_drop(x)
    x = self.out(x)
    if targets is not None:
      loss =self.loss(x,targets)
      met = self.monitor_metrics(x,targets)
      return x,loss,met
    return x,None,{}

     
    
     


      

In [ ]:
import pandas as pd
if __name__=='__main__':
  df = pd.read_csv("IMDB Dataset.csv")
  df['target']=df['sentiment'].apply(lambda x:1 if x=='positive' else 0)
  df['kfold']=-1
  df = df.sample(frac=1).reset_index(drop=True)
  y = df.target
  kf = StratifiedKFold(n_splits=5)
  for f,(t_,v_) in enumerate(kf.split(X=df,y=y)):
    df.loc[v_,'kfold'] = f
  def train_model(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_test = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = BERTDataset(df_train.review.values,df_train.target.values)
    test_dataset = BERTDataset(df_test.review.values,df_test.target.values)
    n_train_steps = int(len(df_train)/32*10)
    model = TextModel(1,num_train_steps=n_train_steps)
    es = tez.callbacks.EarlyStopping(monitor="valid_loss",patience=2,model_path="model.bin")
    model.fit(train_dataset,
              test_dataset,
              epochs=10,
              train_bs=32,
              callbacks=[es])
  train_model(fold=0)


  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

NOTE: This is old Model class and is deprecated. It will no longer be maintained! Please use version > 0.5.1. Its much better and supports multi-gpu training too!


100%|██████████| 625/625 [01:01<00:00, 10.09it/s, accuracy=0.758, loss=0.49, stage=valid]


AttributeError: ignored

In [ ]:
df['target']=df['sentiment'].apply(lambda x:1 if x=='positive' else 0)

In [ ]:
df['kfold']=-1

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
y = df.target

In [ ]:
kf = StratifiedKFold(n_splits=5)
for f,(t_,v_) in enumerate(kf.split(X=df,y=y)):
  df.loc[v_,'kfold'] = f

In [ ]:
df


,review,sentiment,target,kfold
0,I have to agree with Cal-37 it's a great movie...,positive,1,0
1,"Watching ""Kroko"" I would have liked to leave t...",negative,0,0
2,"Going into this movie, I had heard good things...",positive,1,0
3,Recently I borrowed a copy of this mess of a m...,negative,0,0
4,I'm at this very moment debating whether I sho...,negative,0,0
...,...,...,...,...
49995,Bill Paxton stars in and directs this highly o...,positive,1,4
49996,Distortion is a movie that sort of caught me b...,positive,1,4
49997,ELEPHANT WALK may not be the acme of literatur...,positive,1,4
49998,The story takes place on the streets of Sao Pa...,positive,1,4
